Краулим по 700 фотографий перчаток, джинсов и носков для датасета.

In [1]:
from icrawler.builtin import GoogleImageCrawler, BaiduImageCrawler, BingImageCrawler
import glob

In [20]:
clothes_1 = ['gloves', 'jeans', 'socks']
clothes_2 = ['gloves man', 'jeans man', 'socks man']
clothes_3 = ['gloves woman', 'jeans woman', 'socks woman']

def crawler(crawler, clothes):
    for i in clothes:
        work_crawler = crawler(storage = {'root_dir' : 'C:/Users/HUAWEI/OneDrive/Desktop/Crowler/' + i})
        work_crawler.crawl(keyword = i, max_num = 500)

crawler(BingImageCrawler, clothes_1)
crawler(GoogleImageCrawler, clothes_1)
crawler(BaiduImageCrawler, clothes_1)
crawler(BingImageCrawler, clothes_2)
crawler(GoogleImageCrawler, clothes_2)
crawler(BaiduImageCrawler, clothes_2)
crawler(BingImageCrawler, clothes_3)
crawler(GoogleImageCrawler, clothes_3)
crawler(BaiduImageCrawler, clothes_3)

2023-02-23 12:21:23,338 - INFO - icrawler.crawler - start crawling...
2023-02-23 12:21:23,340 - INFO - icrawler.crawler - starting 1 feeder threads...
2023-02-23 12:21:23,343 - INFO - icrawler.crawler - starting 1 parser threads...
2023-02-23 12:21:23,346 - INFO - icrawler.crawler - starting 1 downloader threads...
2023-02-23 12:21:23,717 - INFO - parser - parser-001 is waiting for new page urls
2023-02-23 12:21:24,190 - INFO - parser - parsing result page https://www.bing.com/images/async?q=gloves&first=0
2023-02-23 12:21:24,242 - INFO - downloader - skip downloading file 000001.jpg
2023-02-23 12:21:24,249 - INFO - downloader - skip downloading file 000002.jpg
2023-02-23 12:21:24,253 - INFO - downloader - skip downloading file 000003.jpg
2023-02-23 12:21:24,255 - INFO - downloader - skip downloading file 000004.jpg
2023-02-23 12:21:24,259 - INFO - downloader - skip downloading file 000005.jpg
2023-02-23 12:21:24,261 - INFO - downloader - skip downloading file 000006.jpg
2023-02-23 12:

2023-02-23 12:54:57,414 - INFO - parser - parser-001 is waiting for new page urls
2023-02-23 12:54:57,715 - INFO - parser - parser-001 is waiting for new page urls
2023-02-23 12:54:59,425 - INFO - parser - parser-001 is waiting for new page urls
2023-02-23 12:54:59,725 - INFO - parser - parser-001 is waiting for new page urls


Перемещаем мужские и женские перчатки в папку перчаток, также с джинсами и носками (руками) и делаем равное кол-во файлов в каждой из трех категорий

Проверяем количество фотографий в каждой категории

In [75]:
print(len(glob.glob('C:/Users/HUAWEI/OneDrive/Desktop/Crowler/gloves/*')))
print(len(glob.glob('C:/Users/HUAWEI/OneDrive/Desktop/Crowler/jeans/*')))
print(len(glob.glob('C:/Users/HUAWEI/OneDrive/Desktop/Crowler/socks/*')))

1100
1100
1100


Переходим к самой нейросети

In [120]:
import matplotlib
matplotlib.use("Agg")

In [121]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os

In [122]:
data = []
labels = []

In [123]:
imagePaths = sorted(list(paths.list_images("C:/Users/HUAWEI/OneDrive/Desktop/Crowler/")))

random.shuffle(imagePaths)

In [124]:
print(len(imagePaths))

3300


In [125]:
# цикл по изображениям
for imagePath in imagePaths:
	
	image = cv2.imread(imagePath) # загружаем изображение
	try:
		image = (cv2.resize(image, (32, 32))).flatten() # меняем его разрешение на 32x32 пикселей (без учета соотношения сторон),
																										# сглаживаем его в 32x32x3=3072 пикселей
		label = imagePath.split(os.path.sep)[-2] 	# извлекаем метку класса из пути к изображению (метка класса зависит от имени папки)
		labels.append(label) # обновляем список меток
		data.append(image) # добавляем в список
	except Exception as e:
		pass


In [126]:
print(len(labels))
print(len(data))

2099
2099


In [127]:
# масштабируем интенсивности пикселей в диапазон [0, 1] (Нормализация данных)
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [128]:
# разбиваем данные на обучающую и тестовую выборки, используя 75%
# данных для обучения и оставшиеся 25% для тестирования
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [129]:
# конвертируем метки из целых чисел в векторы (для 2х классов при
# бинарной классификации вам следует использовать функцию Keras
# "to_categorical" вместо "LabelBinarizer" из scikit-learn, которая не возвращает вектор)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [130]:
# определим архитектуру 3072-1024-512-3 с помощью Keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(len(lb.classes_), activation="softmax"))

In [131]:
# инициализируем скорость обучения и общее число эпох
INIT_LR = 0.01
EPOCHS = 120

In [132]:
# компилируем модель, используя SGD как оптимизатор и категориальную
# кросс-энтропию в качестве функции потерь (для бинарной классификации
# следует использовать binary_crossentropy)
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

c:\Users\HUAWEI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [133]:
# обучаем нейросеть
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=1000)

Epoch 1/120
2/2 [==============================] - 1s 258ms/step - loss: 1.2879 - accuracy: 0.3202 - val_loss: 1.3816 - val_accuracy: 0.3219
Epoch 2/120
2/2 [==============================] - 0s 109ms/step - loss: 1.4801 - accuracy: 0.3475 - val_loss: 1.2699 - val_accuracy: 0.3771
Epoch 3/120
2/2 [==============================] - 0s 123ms/step - loss: 1.2124 - accuracy: 0.3666 - val_loss: 1.3517 - val_accuracy: 0.3390
Epoch 4/120
2/2 [==============================] - 0s 117ms/step - loss: 1.2489 - accuracy: 0.3634 - val_loss: 1.1682 - val_accuracy: 0.3410
Epoch 5/120
2/2 [==============================] - 0s 117ms/step - loss: 1.1206 - accuracy: 0.3640 - val_loss: 1.1126 - val_accuracy: 0.3981
Epoch 6/120
2/2 [==============================] - 0s 106ms/step - loss: 1.0947 - accuracy: 0.3971 - val_loss: 1.1583 - val_accuracy: 0.3257
Epoch 7/120
2/2 [==============================] - 0s 113ms/step - loss: 1.1278 - accuracy: 0.3501 - val_loss: 1.1186 - val_accuracy: 0.3981
Epoch 8/120
2

In [134]:
# оцениваем нейросеть
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

17/17 [==============================] - 0s 6ms/step
                                                 precision    recall  f1-score   support

C:/Users/HUAWEI/OneDrive/Desktop/Crowler/gloves       0.62      0.74      0.68       169
 C:/Users/HUAWEI/OneDrive/Desktop/Crowler/jeans       0.91      0.26      0.40       166
 C:/Users/HUAWEI/OneDrive/Desktop/Crowler/socks       0.51      0.75      0.61       190

                                       accuracy                           0.59       525
                                      macro avg       0.68      0.58      0.56       525
                                   weighted avg       0.68      0.59      0.57       525



In [135]:
# строим графики потерь и точности
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('C:/Users/HUAWEI/OneDrive/Desktop/plot.png')
grafic = cv2.imread('C:/Users/HUAWEI/OneDrive/Desktop/plot.png')
cv2.imshow("Image", grafic)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [136]:
# сохраняем модель и метки классов в бинарном представлении на диск
print("[INFO] serializing network and label binarizer...")
model.save("C:/Users/HUAWEI/OneDrive/Desktop/model.h5", save_format="h5")
f = open("C:/Users/HUAWEI/OneDrive/Desktop/label_bin", "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network and label binarizer...


Проверка обученной модели пользовательскими данными

In [137]:
from keras.models import load_model
import argparse
import pickle
import cv2

In [172]:
# Добавляем интерфейс для загрузки изображений напрямую из блокнота
image = cv2.imread('C:/Users/HUAWEI/OneDrive/Desktop/3.jpg')

# загружаем входное изображение, меняем его размер на необходимый и выводим его
#image = cv2.imread(list(uploaded.keys())[0])
output = image.copy()
image = cv2.resize(image, (32, 32))
#cv2.imshow("Image_little", image)

In [173]:
# "Выравниваем" изображение (преобразуем его в вектор. Из 32х32х3 -> 3072)
image = image.flatten()
# масштабируем значения пикселей к диапазону [0, 1]
image = image.astype("float") / 255.0
image = image.reshape((1, image.shape[0]))

In [174]:
# загружаем модель и бинаризатор меток
from keras import models    
print("[INFO] loading network and label binarizer...")
model = models.load_model("model.h5")
lb = pickle.loads(open("label_bin", "rb").read())

[INFO] loading network and label binarizer...


In [175]:
# делаем предсказание на изображении
preds = model.predict(image)
print(preds)

1/1 [==============================] - 0s 66ms/step
[[0.08966423 0.7419275  0.16840824]]


In [176]:
# находим индекс метки класса с наибольшей вероятностью
# соответствия
i = preds.argmax(axis=1)[0]
label = lb.classes_[i]

In [177]:
# Добавляем на исходное изображение метку класса и вероятность прогноза
text = "{}: {:.2f}%".format(label, preds[0][i] * 100)
text = text[text.rfind(chr(47)) + 1:]
cv2.putText(output, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

array([[[222, 227, 228],
        [222, 227, 228],
        [222, 227, 228],
        ...,
        [230, 235, 236],
        [230, 235, 236],
        [230, 235, 236]],

       [[222, 227, 228],
        [222, 227, 228],
        [222, 227, 228],
        ...,
        [231, 235, 236],
        [231, 235, 236],
        [230, 235, 236]],

       [[222, 227, 228],
        [222, 227, 228],
        [222, 227, 228],
        ...,
        [231, 235, 236],
        [231, 235, 236],
        [231, 235, 236]],

       ...,

       [[240, 242, 243],
        [240, 242, 243],
        [240, 242, 243],
        ...,
        [234, 236, 237],
        [234, 236, 237],
        [233, 235, 236]],

       [[241, 243, 244],
        [241, 243, 244],
        [241, 243, 244],
        ...,
        [235, 237, 238],
        [235, 237, 238],
        [236, 238, 239]],

       [[240, 242, 243],
        [240, 242, 243],
        [240, 242, 243],
        ...,
        [235, 237, 238],
        [235, 237, 238],
        [236, 238, 239]]

In [178]:
# показываем выходное изображение
cv2.imwrite('C:/Users/HUAWEI/OneDrive/Desktop/output.jpg', output)

True

In [179]:
cv2.imshow("Image_output", output)
cv2.waitKey(0)
cv2.destroyAllWindows()